In [5]:
import wget
url1 = "https://ftp.pride.ebi.ac.uk/pride/data/archive/2017/02/PXD004732/01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2.raw"
url2 = "https://ftp.pride.ebi.ac.uk/pride/data/archive/2017/02/PXD004732/TUM_first_pool_1_01_01_DDA-1h-R2-tryptic.zip"

In [6]:
wget.download(url1)
wget.download(url2)

'TUM_first_pool_1_01_01_DDA-1h-R2-tryptic.zip'

In [4]:
from zipfile import ZipFile
import pandas as pd
with ZipFile(f'TUM_first_pool_1_01_01_DDA-1h-R2-tryptic.zip', 'r') as zip_file:
  msms = pd.read_csv(zip_file.open('msms.txt'), sep='\t')
# Current PROSIT pipeline does not accomodate modified peptides, so we remove all of the oxidized peptides
msms = msms[msms['Modifications'] == 'Unmodified']

In [5]:
print(msms.head)

<bound method NDFrame.head of                                           Raw file  Scan number  Scan index  \
0      01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        19663       17042   
1      01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        19664       17043   
2      01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        19787       17160   
3      01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        19788       17161   
4      01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        10463        8187   
...                                            ...          ...         ...   
13084  01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        38231       34883   
13085  01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        38348       34996   
13086  01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        38349       34997   
13087  01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        38462       35104   
13088  01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2        38463       35105   

           Sequence  

In [6]:
from fisher_py import RawFile
raw = RawFile('D:/backend/01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2.raw')

In [7]:
from fisher_py import RawFile
raw = RawFile('01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2.raw')
# Get the scan numbers from the msms file and save the scan + info in a dictionary
from fisher_py.data.business import Scan
import numpy as np
scan_mzs = []
scan_ints = []
scan_mass_analyzers = []
scan_collison_energy = []
for scan in msms['Scan number']:
  raw_scan = Scan.from_file(raw._raw_file_access, scan)
  scan_mzs.append(np.array(raw_scan.preferred_masses))
  scan_ints.append(np.array(raw_scan.preferred_intensities))
  scan_mass_analyzers.append(raw_scan.scan_type.split(' + ')[0])
  frag_infos = [f.split(' ')[0] for f in raw_scan.scan_type.split('@')[1:]]
  splits = [[i for i, g in enumerate(f) if g.isnumeric()][0] for f in frag_infos]
  NCEs = [float(frag[split:]) for split, frag in zip(splits, frag_infos)]
  scan_collison_energy.append(NCEs[0])

In [9]:
import spectrum_fundamentals
annotation_df = pd.DataFrame(msms[['Modified sequence', 'Charge', 'Scan number']].values, columns=['MODIFIED_SEQUENCE', 'PRECURSOR_CHARGE', 'SCAN_NUMBER'])
annotation_df['MZ'] = scan_mzs
annotation_df['INTENSITIES'] = scan_ints
annotation_df['MASS_ANALYZER'] = scan_mass_analyzers
annotation_df['COLLISION_ENERGY'] = scan_collison_energy

# from spectrum_fundamentals.mode-strings import maxquant_to_internal
# annotation_df['MODIFIED_SEQUENCE'] = maxquant_to_internal(annotation_df['MODIFIED_SEQUENCE'].values)

# from spectrum_fundamentals.annotation.annotation import annotate_spectra
# annotation = annotate_spectra(annotation_df)

In [ ]:
annotation

In [ ]:
PROSIT_ALHABET = {
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
    "M(ox)": 21,
}
sequence_integer = [[PROSIT_ALHABET[AA] for AA in sequence] for sequence in msms['Sequence']]
precursor_charge_onehot = pd.get_dummies(msms['Charge']).values
collision_energy_aligned_normed = annotation_df['COLLISION_ENERGY']
intensities_raw = annotation['INTENSITIES']

In [ ]:
df = pd.DataFrame(list(zip(sequence_integer, precursor_charge_onehot, collision_energy_aligned_normed, intensities_raw)),
                  columns=['sequence_integer', 'precursor_charge_onehot', 'collision_energy', 'intensities_raw'])
df

In [34]:
import matplotlib.pyplot as plt
from fisher_py import RawFile
from fisher_py.data.business import TraceType
path = r'D:\backend\01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2.raw'
raw_file = RawFile(path=path)

target_mass = 848.36862
mass_tolerance_ppm = 10
rt, i = raw_file.get_chromatogram(target_mass, mass_tolerance_ppm, TraceType.MassRange)
mz, i2, charges, real_rt = raw_file.get_scan_ms1(1)
print(real_rt)

plt.figure()
plt.plot(rt, i)

plt.figure()
plt.plot(mz, i2)

plt.show()

TypeError: No method matches given arguments for IRawData.SelectInstrument: (<class 'int'>, <class 'int'>)

In [ ]:
import pythonnet



In [26]:
from src.helper import *

In [ ]:
read_scan_information(raw_file= RawFileAccess, first_scan_number=1, last_scan_number=100, output_data=True)

In [33]:
path = r'D:\backend\01625b_GA1-TUM_first_pool_1_01_01-DDA-1h-R2.raw'
raw_data = RawFile(path=path)

raw_file.get_instrument_data

TypeError: No method matches given arguments for IRawData.SelectInstrument: (<class 'int'>, <class 'int'>)